## Preprocessing

In [25]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
import pandas as pd
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [26]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df = application_df.drop(['EIN', 'NAME'], axis=1)

In [27]:
# Determine the number of unique values in each column.
column_unique_counts = application_df.nunique()
print(column_unique_counts)

APPLICATION_TYPE            17
AFFILIATION                  6
CLASSIFICATION              71
USE_CASE                     5
ORGANIZATION                 4
STATUS                       2
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
dtype: int64


In [28]:
# Look at APPLICATION_TYPE value counts for binning
application_type_counts = application_df['APPLICATION_TYPE'].value_counts()
print(application_type_counts)

T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: APPLICATION_TYPE, dtype: int64


In [29]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
application_types_to_replace = application_type_counts[application_type_counts < 500].index.tolist()

# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure binning was successful
application_df['APPLICATION_TYPE'].value_counts()

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: APPLICATION_TYPE, dtype: int64

In [30]:
# Look at CLASSIFICATION value counts for binning
classification_counts = application_df['CLASSIFICATION'].value_counts()
print(classification_counts)

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: CLASSIFICATION, Length: 71, dtype: int64


In [31]:
# You may find it helpful to look at CLASSIFICATION value counts >1
classification_counts = application_df['CLASSIFICATION'].value_counts()
classification_counts_filtered = classification_counts[classification_counts > 1]
print(classification_counts_filtered)

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
C2800       95
C7100       75
C1300       58
C1280       50
C1230       36
C1400       34
C7200       32
C2300       32
C1240       30
C8000       20
C7120       18
C1500       16
C1800       15
C6000       15
C1250       14
C8200       11
C1238       10
C1278       10
C1235        9
C1237        9
C7210        7
C2400        6
C1720        6
C4100        6
C1257        5
C1600        5
C1260        3
C2710        3
C0           3
C3200        2
C1234        2
C1246        2
C1267        2
C1256        2
Name: CLASSIFICATION, dtype: int64


In [32]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
classifications_to_replace = classification_counts[classification_counts < 500].index.tolist()

# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")

# Check to make sure binning was successful
application_df['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
Other     1484
C7000      777
Name: CLASSIFICATION, dtype: int64

In [33]:
# Convert categorical data to numeric with `pd.get_dummies`
# Select the columns with categorical variables
categorical_columns = ['APPLICATION_TYPE', 'AFFILIATION', 'CLASSIFICATION', 'USE_CASE', 'ORGANIZATION', 'STATUS', 'INCOME_AMT', 'SPECIAL_CONSIDERATIONS']

# Convert categorical variables to numeric
application_df_encoded = pd.get_dummies(application_df, columns=categorical_columns)

# Display the encoded DataFrame
print(application_df_encoded.head())

   ASK_AMT  IS_SUCCESSFUL  APPLICATION_TYPE_Other  APPLICATION_TYPE_T10  \
0     5000              1                       0                     1   
1   108590              1                       0                     0   
2     5000              0                       0                     0   
3     6692              1                       0                     0   
4   142590              1                       0                     0   

   APPLICATION_TYPE_T19  APPLICATION_TYPE_T3  APPLICATION_TYPE_T4  \
0                     0                    0                    0   
1                     0                    1                    0   
2                     0                    0                    0   
3                     0                    1                    0   
4                     0                    1                    0   

   APPLICATION_TYPE_T5  APPLICATION_TYPE_T6  APPLICATION_TYPE_T7  ...  \
0                    0                    0                  

In [34]:
# Split our preprocessed data into our features and target arrays
X = application_df_encoded.drop('IS_SUCCESSFUL', axis=1)
y = application_df_encoded['IS_SUCCESSFUL']

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Print the shapes of the training and testing sets
print("X_train shape:", X_train.shape)
print("y_train shape:", y_train.shape)
print("X_test shape:", X_test.shape)
print("y_test shape:", y_test.shape)

X_train shape: (27439, 45)
y_train shape: (27439,)
X_test shape: (6860, 45)
y_test shape: (6860,)


In [35]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Compile, Train and Evaluate the Model

In [36]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(Dense(units=32, input_dim=X_train_scaled.shape[1], activation='relu'))

# Second hidden layer
nn.add(Dense(units=16, activation='relu'))

# Output layer
nn.add(Dense(units=1, activation='sigmoid'))

# Check the structure of the model
nn.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_7 (Dense)             (None, 32)                1472      
                                                                 
 dense_8 (Dense)             (None, 16)                528       
                                                                 
 dense_9 (Dense)             (None, 1)                 17        
                                                                 
Total params: 2,017
Trainable params: 2,017
Non-trainable params: 0
_________________________________________________________________


In [37]:
# Compile the model
nn.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [38]:
# Train the model
model_history = nn.fit(X_train_scaled, y_train, epochs=50, batch_size=32, verbose=1)

Epoch 1/50
858/858 [==============================] - 3s 3ms/step - loss: 0.5796 - accuracy: 0.7131
Epoch 2/50
858/858 [==============================] - 2s 2ms/step - loss: 0.5539 - accuracy: 0.7296
Epoch 3/50
858/858 [==============================] - 2s 2ms/step - loss: 0.5510 - accuracy: 0.7307
Epoch 4/50
858/858 [==============================] - 2s 2ms/step - loss: 0.5487 - accuracy: 0.7308
Epoch 5/50
858/858 [==============================] - 2s 2ms/step - loss: 0.5480 - accuracy: 0.7317
Epoch 6/50
858/858 [==============================] - 2s 2ms/step - loss: 0.5474 - accuracy: 0.7307
Epoch 7/50
858/858 [==============================] - 2s 2ms/step - loss: 0.5460 - accuracy: 0.7329
Epoch 8/50
858/858 [==============================] - 2s 2ms/step - loss: 0.5462 - accuracy: 0.7320
Epoch 9/50
858/858 [==============================] - 2s 3ms/step - loss: 0.5449 - accuracy: 0.7317
Epoch 10/50
858/858 [==============================] - 2s 2ms/step - loss: 0.5446 - accuracy: 0.7340

In [39]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

215/215 - 0s - loss: 0.5530 - accuracy: 0.7262 - 372ms/epoch - 2ms/step
Loss: 0.5530235171318054, Accuracy: 0.7262390851974487


# Compile the model using keras-tuner to find the best model

In [40]:
! pip install keras-tuner

In [41]:
import keras_tuner as kt

# Define the model
def create_model(hp):
    model = Sequential()
    activation_choice = hp.Choice('activation', ['relu', 'sigmoid', 'tanh'])
    model.add(Dense(units=hp.Int('first_units', min_value=1, max_value=12, step=2), activation=activation_choice, input_dim = X_train_scaled.shape[1]))

    for i in range(hp.Int("num_layers", 1, 6)):
        model.add(Dense(units=hp.Int(f'units_{i}', min_value=1, max_value=12, step=2), activation=activation_choice))

    model.add(Dense(units=1, activation='sigmoid'))
    model.compile(loss="binary_crossentropy", optimizer='adam', metrics=["accuracy"])

    return model

# Create the tuner
tuner = kt.Hyperband(create_model, objective='val_accuracy', max_epochs=20, hyperband_iterations=2)

# Perform the hyperparameter search
tuner.search(X_train_scaled, y_train, epochs=20, validation_data=(X_test_scaled, y_test))

# Get the best hyperparameters
best_hyperparameters = tuner.get_best_hyperparameters(1)[0].values

# Get the best model
best_model = tuner.get_best_models(1)[0]

# Evaluate the best model
test_loss, test_accuracy = best_model.evaluate(X_test_scaled, y_test)
print("Test Loss:", test_loss)
print("Test Accuracy:", test_accuracy)


215/215 [==============================] - 1s 2ms/step - loss: 0.5593 - accuracy: 0.7306
Test Loss: 0.5593209266662598
Test Accuracy: 0.7306122183799744


In [42]:
tuner.results_summary(2)

Results summary
Results in ./untitled_project
Showing 2 best trials
Objective(name="val_accuracy", direction="max")

Trial 0019 summary
Hyperparameters:
activation: tanh
first_units: 11
num_layers: 5
units_0: 3
units_1: 9
units_2: 11
units_3: 9
units_4: 9
units_5: 9
tuner/epochs: 20
tuner/initial_epoch: 7
tuner/bracket: 1
tuner/round: 1
tuner/trial_id: 0016
Score: 0.7306122183799744

Trial 0041 summary
Hyperparameters:
activation: tanh
first_units: 9
num_layers: 4
units_0: 11
units_1: 11
units_2: 5
units_3: 11
units_4: 5
units_5: 1
tuner/epochs: 20
tuner/initial_epoch: 7
tuner/bracket: 2
tuner/round: 2
tuner/trial_id: 0039
Score: 0.7295918464660645


In [43]:
# Export our model to HDF5 file
nn.save("AlphabetSoupCharity_Optimization.h5")